## Ενσωματώσεις

Στο προηγούμενο παράδειγμά μας, εργαστήκαμε με διανύσματα υψηλής διάστασης bag-of-words με μήκος `vocab_size`, και μετατρέπαμε ρητά από διανύσματα χαμηλής διάστασης θέσης σε αραιή αναπαράσταση one-hot. Αυτή η αναπαράσταση one-hot δεν είναι αποδοτική από άποψη μνήμης, επιπλέον, κάθε λέξη αντιμετωπίζεται ανεξάρτητα από τις άλλες, δηλαδή τα διανύσματα one-hot δεν εκφράζουν καμία σημασιολογική ομοιότητα μεταξύ των λέξεων.

Σε αυτήν την ενότητα, θα συνεχίσουμε να εξερευνούμε το σύνολο δεδομένων **News AG**. Για να ξεκινήσουμε, ας φορτώσουμε τα δεδομένα και ας πάρουμε κάποιους ορισμούς από το προηγούμενο notebook.


In [1]:
import torch
import torchtext
import numpy as np
from torchnlp import *
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_size = len(vocab)
print("Vocab size = ",vocab_size)

Loading dataset...


d:\WORK\ai-for-beginners\5-NLP\14-Embeddings\data\train.csv: 29.5MB [00:01, 18.8MB/s]                            
d:\WORK\ai-for-beginners\5-NLP\14-Embeddings\data\test.csv: 1.86MB [00:00, 11.2MB/s]                          


Building vocab...
Vocab size =  95812


## Τι είναι η ενσωμάτωση;

Η ιδέα της **ενσωμάτωσης** είναι να αναπαρασταθούν οι λέξεις με πυκνά διανύσματα χαμηλότερης διάστασης, τα οποία με κάποιο τρόπο αντανακλούν τη σημασιολογική έννοια μιας λέξης. Αργότερα θα συζητήσουμε πώς να δημιουργήσουμε ουσιαστικές ενσωματώσεις λέξεων, αλλά προς το παρόν ας σκεφτούμε την ενσωμάτωση ως έναν τρόπο μείωσης της διάστασης ενός διανύσματος λέξης.

Έτσι, η στρώση ενσωμάτωσης θα παίρνει μια λέξη ως είσοδο και θα παράγει ένα διανυσματικό αποτέλεσμα με καθορισμένο `embedding_size`. Με μια έννοια, είναι πολύ παρόμοια με τη στρώση `Linear`, αλλά αντί να λαμβάνει ένα one-hot κωδικοποιημένο διάνυσμα, θα μπορεί να λαμβάνει έναν αριθμό λέξης ως είσοδο.

Χρησιμοποιώντας τη στρώση ενσωμάτωσης ως την πρώτη στρώση στο δίκτυό μας, μπορούμε να μεταβούμε από το μοντέλο bag-of-words στο μοντέλο **embedding bag**, όπου πρώτα μετατρέπουμε κάθε λέξη στο κείμενό μας στην αντίστοιχη ενσωμάτωσή της και στη συνέχεια υπολογίζουμε κάποια συνάρτηση συσσωμάτωσης πάνω σε όλες αυτές τις ενσωματώσεις, όπως `sum`, `average` ή `max`.

![Εικόνα που δείχνει έναν ταξινομητή ενσωμάτωσης για πέντε λέξεις ακολουθίας.](../../../../../translated_images/el/embedding-classifier-example.b77f021a7ee67eee.webp)

Το νευρωνικό δίκτυο ταξινομητή μας θα ξεκινά με τη στρώση ενσωμάτωσης, στη συνέχεια τη στρώση συσσωμάτωσης και έναν γραμμικό ταξινομητή στην κορυφή:


In [2]:
class EmbedClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.fc = torch.nn.Linear(embed_dim, num_class)

    def forward(self, x):
        x = self.embedding(x)
        x = torch.mean(x,dim=1)
        return self.fc(x)

### Αντιμετώπιση μεταβλητού μεγέθους ακολουθίας

Ως αποτέλεσμα αυτής της αρχιτεκτονικής, τα minibatches για το δίκτυό μας θα πρέπει να δημιουργούνται με συγκεκριμένο τρόπο. Στην προηγούμενη ενότητα, όταν χρησιμοποιούσαμε το bag-of-words, όλοι οι BoW tensors σε ένα minibatch είχαν ίσο μέγεθος `vocab_size`, ανεξάρτητα από το πραγματικό μήκος της ακολουθίας κειμένου μας. Μόλις περάσουμε στις ενσωματώσεις λέξεων, θα καταλήξουμε με μεταβλητό αριθμό λέξεων σε κάθε δείγμα κειμένου, και όταν συνδυάζουμε αυτά τα δείγματα σε minibatches θα πρέπει να εφαρμόσουμε κάποια συμπλήρωση.

Αυτό μπορεί να γίνει χρησιμοποιώντας την ίδια τεχνική της παροχής της συνάρτησης `collate_fn` στην πηγή δεδομένων:


In [3]:
def padify(b):
    # b is the list of tuples of length batch_size
    #   - first element of a tuple = label, 
    #   - second = feature (text sequence)
    # build vectorized sequence
    v = [encode(x[1]) for x in b]
    # first, compute max length of a sequence in this minibatch
    l = max(map(len,v))
    return ( # tuple of two tensors - labels and features
        torch.LongTensor([t[0]-1 for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v])
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=padify, shuffle=True)

### Εκπαίδευση του ταξινομητή ενσωμάτωσης

Τώρα που έχουμε ορίσει τον κατάλληλο φορτωτή δεδομένων, μπορούμε να εκπαιδεύσουμε το μοντέλο χρησιμοποιώντας τη συνάρτηση εκπαίδευσης που ορίσαμε στην προηγούμενη ενότητα:


In [4]:
net = EmbedClassifier(vocab_size,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=1, epoch_size=25000)

3200: acc=0.6415625
6400: acc=0.6865625
9600: acc=0.7103125
12800: acc=0.726953125
16000: acc=0.739375
19200: acc=0.75046875
22400: acc=0.7572321428571429


(0.889799795315499, 0.7623160588611644)

> **Σημείωση**: Εδώ εκπαιδεύουμε μόνο για 25k εγγραφές (λιγότερο από μία πλήρη εποχή) για λόγους χρόνου, αλλά μπορείτε να συνεχίσετε την εκπαίδευση, να γράψετε μια συνάρτηση για εκπαίδευση για αρκετές εποχές και να πειραματιστείτε με την παράμετρο του ρυθμού μάθησης για να επιτύχετε μεγαλύτερη ακρίβεια. Θα πρέπει να μπορείτε να φτάσετε σε ακρίβεια περίπου 90%.


### Επίπεδο EmbeddingBag και Αναπαράσταση Ακολουθιών Μεταβλητού Μήκους

Στην προηγούμενη αρχιτεκτονική, έπρεπε να συμπληρώσουμε όλες τις ακολουθίες ώστε να έχουν το ίδιο μήκος για να τις εντάξουμε σε ένα minibatch. Αυτός δεν είναι ο πιο αποδοτικός τρόπος για να αναπαραστήσουμε ακολουθίες μεταβλητού μήκους - μια άλλη προσέγγιση θα ήταν να χρησιμοποιήσουμε έναν **διάνυσμα μετατοπίσεων (offset)**, το οποίο θα περιείχε τις μετατοπίσεις όλων των ακολουθιών που αποθηκεύονται σε ένα μεγάλο διάνυσμα.

![Εικόνα που δείχνει την αναπαράσταση ακολουθιών με μετατοπίσεις](../../../../../translated_images/el/offset-sequence-representation.eb73fcefb29b46ee.webp)

> **Σημείωση**: Στην παραπάνω εικόνα, δείχνουμε μια ακολουθία χαρακτήρων, αλλά στο παράδειγμά μας δουλεύουμε με ακολουθίες λέξεων. Ωστόσο, η γενική αρχή της αναπαράστασης ακολουθιών με διάνυσμα μετατοπίσεων παραμένει η ίδια.

Για να δουλέψουμε με την αναπαράσταση μετατοπίσεων, χρησιμοποιούμε το [`EmbeddingBag`](https://pytorch.org/docs/stable/generated/torch.nn.EmbeddingBag.html) επίπεδο. Είναι παρόμοιο με το `Embedding`, αλλά δέχεται ως είσοδο ένα διάνυσμα περιεχομένου και ένα διάνυσμα μετατοπίσεων, και περιλαμβάνει επίσης ένα επίπεδο μέσου όρου, το οποίο μπορεί να είναι `mean`, `sum` ή `max`.

Ακολουθεί ένα τροποποιημένο δίκτυο που χρησιμοποιεί το `EmbeddingBag`:


In [5]:
class EmbedClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.EmbeddingBag(vocab_size, embed_dim)
        self.fc = torch.nn.Linear(embed_dim, num_class)

    def forward(self, text, off):
        x = self.embedding(text, off)
        return self.fc(x)

Για να προετοιμάσουμε το σύνολο δεδομένων για εκπαίδευση, πρέπει να παρέχουμε μια συνάρτηση μετατροπής που θα προετοιμάσει το διανύσμα μετατόπισης:


In [6]:
def offsetify(b):
    # first, compute data tensor from all sequences
    x = [torch.tensor(encode(t[1])) for t in b]
    # now, compute the offsets by accumulating the tensor of sequence lengths
    o = [0] + [len(t) for t in x]
    o = torch.tensor(o[:-1]).cumsum(dim=0)
    return ( 
        torch.LongTensor([t[0]-1 for t in b]), # labels
        torch.cat(x), # text 
        o
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=offsetify, shuffle=True)

Σημειώστε ότι, σε αντίθεση με όλα τα προηγούμενα παραδείγματα, το δίκτυό μας τώρα δέχεται δύο παραμέτρους: το διανύσμα δεδομένων και το διανύσμα μετατόπισης, τα οποία έχουν διαφορετικά μεγέθη. Παρομοίως, ο φορτωτής δεδομένων μας παρέχει επίσης 3 τιμές αντί για 2: τόσο τα διανύσματα κειμένου όσο και τα διανύσματα μετατόπισης παρέχονται ως χαρακτηριστικά. Επομένως, πρέπει να προσαρμόσουμε ελαφρώς τη συνάρτηση εκπαίδευσης μας για να το λάβουμε υπόψη:


In [7]:
net = EmbedClassifier(vocab_size,32,len(classes)).to(device)

def train_epoch_emb(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.CrossEntropyLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    loss_fn = loss_fn.to(device)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,text,off in dataloader:
        optimizer.zero_grad()
        labels,text,off = labels.to(device), text.to(device), off.to(device)
        out = net(text, off)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count


train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6153125
6400: acc=0.6615625
9600: acc=0.6932291666666667
12800: acc=0.715078125
16000: acc=0.7270625
19200: acc=0.7382291666666667
22400: acc=0.7486160714285715


(22.771553103007037, 0.7551983365323096)

## Σημασιολογικές Ενσωματώσεις: Word2Vec

Στο προηγούμενο παράδειγμα, το επίπεδο ενσωμάτωσης του μοντέλου έμαθε να αντιστοιχεί λέξεις σε διανυσματικές αναπαραστάσεις, ωστόσο, αυτή η αναπαράσταση δεν είχε ιδιαίτερη σημασιολογική αξία. Θα ήταν χρήσιμο να μάθουμε μια τέτοια διανυσματική αναπαράσταση, όπου παρόμοιες λέξεις ή συνώνυμα θα αντιστοιχούσαν σε διανύσματα που βρίσκονται κοντά μεταξύ τους με βάση κάποια απόσταση διανυσμάτων (π.χ. ευκλείδεια απόσταση).

Για να το πετύχουμε αυτό, πρέπει να προεκπαιδεύσουμε το μοντέλο ενσωμάτωσης σε μια μεγάλη συλλογή κειμένων με συγκεκριμένο τρόπο. Ένας από τους πρώτους τρόπους εκπαίδευσης σημασιολογικών ενσωματώσεων ονομάζεται [Word2Vec](https://en.wikipedia.org/wiki/Word2vec). Βασίζεται σε δύο κύριες αρχιτεκτονικές που χρησιμοποιούνται για την παραγωγή κατανεμημένων αναπαραστάσεων λέξεων:

 - **Συνεχές σακίδιο λέξεων** (CBoW) — σε αυτή την αρχιτεκτονική, εκπαιδεύουμε το μοντέλο να προβλέπει μια λέξη από το περιβάλλον της. Δεδομένου του ngram $(W_{-2},W_{-1},W_0,W_1,W_2)$, ο στόχος του μοντέλου είναι να προβλέψει το $W_0$ από το $(W_{-2},W_{-1},W_1,W_2)$.
 - **Συνεχές skip-gram** — είναι το αντίθετο του CBoW. Το μοντέλο χρησιμοποιεί το παράθυρο των λέξεων του περιβάλλοντος για να προβλέψει την τρέχουσα λέξη.

Το CBoW είναι ταχύτερο, ενώ το skip-gram είναι πιο αργό, αλλά αποδίδει καλύτερα στην αναπαράσταση σπάνιων λέξεων.

![Εικόνα που δείχνει τους αλγορίθμους CBoW και Skip-Gram για τη μετατροπή λέξεων σε διανύσματα.](../../../../../translated_images/el/example-algorithms-for-converting-words-to-vectors.fbe9207a726922f6.webp)

Για να πειραματιστούμε με την ενσωμάτωση word2vec που έχει προεκπαιδευτεί στο σύνολο δεδομένων Google News, μπορούμε να χρησιμοποιήσουμε τη βιβλιοθήκη **gensim**. Παρακάτω βρίσκουμε τις λέξεις που είναι πιο παρόμοιες με τη λέξη 'neural'.

> **Σημείωση:** Όταν δημιουργείτε για πρώτη φορά διανύσματα λέξεων, η λήψη τους μπορεί να πάρει κάποιο χρόνο!


In [8]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

In [9]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

neuronal -> 0.7804799675941467
neurons -> 0.7326500415802002
neural_circuits -> 0.7252851724624634
neuron -> 0.7174385190010071
cortical -> 0.6941086649894714
brain_circuitry -> 0.6923246383666992
synaptic -> 0.6699118614196777
neural_circuitry -> 0.6638563275337219
neurochemical -> 0.6555314064025879
neuronal_activity -> 0.6531826257705688


Μπορούμε επίσης να υπολογίσουμε ενσωματώσεις διανυσμάτων από τη λέξη, για να χρησιμοποιηθούν στην εκπαίδευση του μοντέλου ταξινόμησης (δείχνουμε μόνο τα πρώτα 20 στοιχεία του διανύσματος για σαφήνεια):


In [10]:
w2v.word_vec('play')[:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

Το σπουδαίο με τις σημασιολογικές ενσωματώσεις είναι ότι μπορείτε να χειριστείτε την κωδικοποίηση των διανυσμάτων για να αλλάξετε τη σημασιολογία. Για παράδειγμα, μπορούμε να ζητήσουμε να βρούμε μια λέξη, της οποίας η διανυσματική αναπαράσταση θα είναι όσο το δυνατόν πιο κοντά στις λέξεις *βασιλιάς* και *γυναίκα*, και όσο το δυνατόν πιο μακριά από τη λέξη *άντρας*:


In [10]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118192911148071)

Τόσο το CBoW όσο και τα Skip-Grams είναι "προβλεπτικά" embeddings, καθώς λαμβάνουν υπόψη μόνο τοπικά συμφραζόμενα. Το Word2Vec δεν εκμεταλλεύεται το παγκόσμιο συμφραζόμενο.

**Το FastText** βασίζεται στο Word2Vec μαθαίνοντας διανυσματικές αναπαραστάσεις για κάθε λέξη και τα χαρακτήρα n-grams που βρίσκονται μέσα σε κάθε λέξη. Οι τιμές των αναπαραστάσεων στη συνέχεια υπολογίζονται ως μέσος όρος σε ένα διάνυσμα σε κάθε βήμα εκπαίδευσης. Παρόλο που αυτό προσθέτει αρκετό επιπλέον υπολογισμό κατά την προεκπαίδευση, επιτρέπει στα word embeddings να κωδικοποιούν πληροφορίες υπολέξεων.

Μια άλλη μέθοδος, το **GloVe**, αξιοποιεί την ιδέα του πίνακα συν-εμφάνισης, χρησιμοποιώντας νευρωνικές μεθόδους για να αποσυνθέσει τον πίνακα συν-εμφάνισης σε πιο εκφραστικά και μη γραμμικά διανύσματα λέξεων.

Μπορείτε να πειραματιστείτε με το παράδειγμα αλλάζοντας τα embeddings σε FastText και GloVe, καθώς το gensim υποστηρίζει διάφορα μοντέλα ενσωμάτωσης λέξεων.


## Χρήση Προεκπαιδευμένων Ενσωματώσεων στο PyTorch

Μπορούμε να τροποποιήσουμε το παραπάνω παράδειγμα ώστε να προ-γεμίσουμε τον πίνακα στο επίπεδο ενσωμάτωσης μας με σημασιολογικές ενσωματώσεις, όπως το Word2Vec. Πρέπει να λάβουμε υπόψη ότι τα λεξιλόγια των προεκπαιδευμένων ενσωματώσεων και του κειμενικού μας σώματος πιθανότατα δεν θα ταιριάζουν, οπότε θα αρχικοποιήσουμε τα βάρη για τις λέξεις που λείπουν με τυχαίες τιμές:


In [11]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

net = EmbedClassifier(vocab_size,embed_size,len(classes))

print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab.get_itos()):
    try:
        net.embedding.weight[i].data = torch.tensor(w2v.get_vector(w))
        found+=1
    except:
        net.embedding.weight[i].data = torch.normal(0.0,1.0,(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")
net = net.to(device)

Embedding size: 300
Populating matrix, this will take some time...Done, found 41080 words, 54732 words missing


Τώρα ας εκπαιδεύσουμε το μοντέλο μας. Σημειώστε ότι ο χρόνος που απαιτείται για την εκπαίδευση του μοντέλου είναι σημαντικά μεγαλύτερος από το προηγούμενο παράδειγμα, λόγω του μεγαλύτερου μεγέθους της στρώσης ενσωμάτωσης, και επομένως πολύ μεγαλύτερου αριθμού παραμέτρων. Επίσης, εξαιτίας αυτού, μπορεί να χρειαστεί να εκπαιδεύσουμε το μοντέλο μας σε περισσότερα παραδείγματα αν θέλουμε να αποφύγουμε την υπερπροσαρμογή.


In [12]:
train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6359375
6400: acc=0.68109375
9600: acc=0.7067708333333333
12800: acc=0.723671875
16000: acc=0.73625
19200: acc=0.7463541666666667
22400: acc=0.7560714285714286


(214.1013875559821, 0.7626759436980166)

Στην περίπτωσή μας, δεν παρατηρούμε μεγάλη αύξηση στην ακρίβεια, κάτι που πιθανότατα οφείλεται στις αρκετά διαφορετικές λεξιλογικές δομές.  
Για να ξεπεράσουμε το πρόβλημα των διαφορετικών λεξιλογίων, μπορούμε να χρησιμοποιήσουμε μία από τις παρακάτω λύσεις:  
* Επανεκπαίδευση του μοντέλου word2vec στο δικό μας λεξιλόγιο  
* Φόρτωση του dataset μας με το λεξιλόγιο από το προεκπαιδευμένο μοντέλο word2vec. Το λεξιλόγιο που χρησιμοποιείται για τη φόρτωση του dataset μπορεί να καθοριστεί κατά τη διαδικασία φόρτωσης.  

Η δεύτερη προσέγγιση φαίνεται ευκολότερη, ειδικά επειδή το πλαίσιο `torchtext` της PyTorch περιέχει ενσωματωμένη υποστήριξη για embeddings. Μπορούμε, για παράδειγμα, να δημιουργήσουμε λεξιλόγιο βασισμένο στο GloVe με τον εξής τρόπο:


In [14]:
vocab = torchtext.vocab.GloVe(name='6B', dim=50)

100%|█████████▉| 399999/400000 [00:15<00:00, 25411.14it/s]


Το φορτωμένο λεξιλόγιο έχει τις εξής βασικές λειτουργίες:
* Το λεξικό `vocab.stoi` μας επιτρέπει να μετατρέψουμε μια λέξη στον δείκτη της στο λεξικό
* Το `vocab.itos` κάνει το αντίθετο - μετατρέπει έναν αριθμό σε λέξη
* Το `vocab.vectors` είναι ο πίνακας των διανυσμάτων ενσωμάτωσης, οπότε για να πάρουμε την ενσωμάτωση μιας λέξης `s` πρέπει να χρησιμοποιήσουμε `vocab.vectors[vocab.stoi[s]]`

Ακολουθεί ένα παράδειγμα χειρισμού ενσωματώσεων για να δείξουμε την εξίσωση **kind-man+woman = queen** (έπρεπε να προσαρμόσω λίγο τον συντελεστή για να λειτουργήσει):


In [15]:
# get the vector corresponding to kind-man+woman
qvec = vocab.vectors[vocab.stoi['king']]-vocab.vectors[vocab.stoi['man']]+1.3*vocab.vectors[vocab.stoi['woman']]
# find the index of the closest embedding vector 
d = torch.sum((vocab.vectors-qvec)**2,dim=1)
min_idx = torch.argmin(d)
# find the corresponding word
vocab.itos[min_idx]

'queen'

Για να εκπαιδεύσουμε τον ταξινομητή χρησιμοποιώντας αυτές τις ενσωματώσεις, πρέπει πρώτα να κωδικοποιήσουμε το σύνολο δεδομένων μας χρησιμοποιώντας το λεξιλόγιο του GloVe:


In [16]:
def offsetify(b):
    # first, compute data tensor from all sequences
    x = [torch.tensor(encode(t[1],voc=vocab)) for t in b] # pass the instance of vocab to encode function!
    # now, compute the offsets by accumulating the tensor of sequence lengths
    o = [0] + [len(t) for t in x]
    o = torch.tensor(o[:-1]).cumsum(dim=0)
    return ( 
        torch.LongTensor([t[0]-1 for t in b]), # labels
        torch.cat(x), # text 
        o
    )

Όπως είδαμε παραπάνω, όλες οι ενσωματώσεις διανυσμάτων αποθηκεύονται στον πίνακα `vocab.vectors`. Αυτό καθιστά εξαιρετικά εύκολη τη φόρτωση αυτών των βαρών στα βάρη της στρώσης ενσωμάτωσης χρησιμοποιώντας απλή αντιγραφή:


In [17]:
net = EmbedClassifier(len(vocab),len(vocab.vectors[0]),len(classes))
net.embedding.weight.data = vocab.vectors
net = net.to(device)

Τώρα ας εκπαιδεύσουμε το μοντέλο μας και να δούμε αν θα έχουμε καλύτερα αποτελέσματα:


In [18]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=offsetify, shuffle=True)
train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6271875
6400: acc=0.68078125
9600: acc=0.7030208333333333
12800: acc=0.71984375
16000: acc=0.7346875
19200: acc=0.7455729166666667
22400: acc=0.7529464285714286


(35.53972978646833, 0.7575175943698017)

Ένας από τους λόγους που δεν παρατηρούμε σημαντική αύξηση στην ακρίβεια είναι το γεγονός ότι ορισμένες λέξεις από το σύνολο δεδομένων μας λείπουν από το προεκπαιδευμένο λεξιλόγιο του GloVe και, επομένως, ουσιαστικά αγνοούνται. Για να ξεπεράσουμε αυτό το γεγονός, μπορούμε να εκπαιδεύσουμε τις δικές μας ενσωματώσεις στο σύνολο δεδομένων μας.


## Εννοιολογικές Ενσωματώσεις

Ένας βασικός περιορισμός των παραδοσιακών προεκπαιδευμένων αναπαραστάσεων ενσωματώσεων, όπως το Word2Vec, είναι το πρόβλημα της αποσαφήνισης της σημασίας των λέξεων. Παρόλο που οι προεκπαιδευμένες ενσωματώσεις μπορούν να αποτυπώσουν μέρος της σημασίας των λέξεων στο πλαίσιο τους, κάθε πιθανή σημασία μιας λέξης κωδικοποιείται στην ίδια ενσωμάτωση. Αυτό μπορεί να προκαλέσει προβλήματα σε μοντέλα που βασίζονται σε αυτά, καθώς πολλές λέξεις, όπως η λέξη 'play', έχουν διαφορετικές σημασίες ανάλογα με το πλαίσιο στο οποίο χρησιμοποιούνται.

Για παράδειγμα, η λέξη 'play' στις παρακάτω δύο προτάσεις έχει αρκετά διαφορετική σημασία:
- Πήγα σε μια **παράσταση** στο θέατρο.
- Ο Γιάννης θέλει να **παίξει** με τους φίλους του.

Οι προεκπαιδευμένες ενσωματώσεις που αναφέρθηκαν παραπάνω αντιπροσωπεύουν και τις δύο αυτές σημασίες της λέξης 'play' στην ίδια ενσωμάτωση. Για να ξεπεράσουμε αυτόν τον περιορισμό, πρέπει να δημιουργήσουμε ενσωματώσεις βασισμένες στο **γλωσσικό μοντέλο**, το οποίο έχει εκπαιδευτεί σε ένα μεγάλο σώμα κειμένου και *γνωρίζει* πώς οι λέξεις μπορούν να συνδυαστούν σε διαφορετικά πλαίσια. Η συζήτηση για τις εννοιολογικές ενσωματώσεις είναι εκτός του πεδίου αυτού του οδηγού, αλλά θα επανέλθουμε σε αυτές όταν μιλήσουμε για γλωσσικά μοντέλα στην επόμενη ενότητα.



---

**Αποποίηση Ευθύνης**:  
Αυτό το έγγραφο έχει μεταφραστεί χρησιμοποιώντας την υπηρεσία αυτόματης μετάφρασης AI [Co-op Translator](https://github.com/Azure/co-op-translator). Παρόλο που καταβάλλουμε προσπάθειες για ακρίβεια, παρακαλούμε να έχετε υπόψη ότι οι αυτόματες μεταφράσεις ενδέχεται να περιέχουν σφάλματα ή ανακρίβειες. Το πρωτότυπο έγγραφο στη μητρική του γλώσσα θα πρέπει να θεωρείται η αυθεντική πηγή. Για κρίσιμες πληροφορίες, συνιστάται επαγγελματική ανθρώπινη μετάφραση. Δεν φέρουμε ευθύνη για τυχόν παρεξηγήσεις ή εσφαλμένες ερμηνείες που προκύπτουν από τη χρήση αυτής της μετάφρασης.
